In [1]:
from interpretml_tools import *

from interpret.glassbox import ExplainableBoostingClassifier, ExplainableBoostingRegressor, merge_ebms

import pandas as pd
import numpy as np  
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

# Loading dataset
### (German)

In [2]:
# Load German Credit Dataset
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.data"
columns = [
    'checking_status', 'duration', 'credit_history', 'purpose', 'credit_amount',
    'savings_account', 'employment', 'installment_rate', 'personal_status_sex',
    'other_debtors', 'present_residence', 'property', 'age', 'other_installment_plans',
    'housing', 'existing_credits', 'job', 'num_maintenance', 'telephone', 'foreign_worker', 'target'
]

df = pd.read_csv(url, sep=' ', names=columns, header=None)

# Preprocessing
# Create binary sex feature (Male=1, Female=0)
df['sex'] = df['personal_status_sex'].apply(lambda x: 'male' if x in ['A91', 'A93', 'A94'] else 'female')

# Convert target to binary (Good credit=1, Bad credit=0)
df['target'] = df['target'].replace({1: 1, 2: 0})

features = df.columns.tolist()
features.remove('target')

X = df[features]
y = df['target']

# Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Training baseline models

In [3]:
male_model = ExplainableBoostingClassifier(feature_names=X.columns.tolist())
male_model.fit(X_train[X_train['sex'] == 'male'], y_train[X_train['sex'] == 'male'])

female_model = ExplainableBoostingClassifier(feature_names=X.columns.tolist())
female_model.fit(X_train[X_train['sex'] == 'female'], y_train[X_train['sex'] == 'female'])

normal_model = ExplainableBoostingClassifier(feature_names=X.columns.tolist())
normal_model.fit(X_train, y_train)

print("done")

done


In [4]:
ff_model = CombinedEBM([male_model, female_model], [0.5, 0.5])

In [5]:
combined = merge_ebms([male_model, female_model])

# Displaying with custom EBMVisualizer

In [6]:
%matplotlib widget
plt.ioff()
visualizer = InterpretmlEBMVisualizer([male_model, female_model, normal_model, ff_model, combined], ["Male Model", "Female Model", "Normal Model", "50-50 Model", "Combined"])
visualizer.show()

# Group Performance Plots

In [3]:
male_model = ExplainableBoostingClassifier(feature_names=X.columns.tolist())
male_model.fit(X_train[X_train['sex'] == 'male'], y_train[X_train['sex'] == 'male'])

female_model = ExplainableBoostingClassifier(feature_names=X.columns.tolist())
female_model.fit(X_train[X_train['sex'] == 'female'], y_train[X_train['sex'] == 'female'])

female_model_eps = ExplainableBoostingClassifier(feature_names=X.columns.tolist())
eps = 1e-10
female_model_eps.fit(X_train, y_train, sample_weight=X_train['sex'].map(lambda x: eps if x == 'male' else 1 - eps))

male_model_eps = ExplainableBoostingClassifier(feature_names=X.columns.tolist())
male_model_eps.fit(X_train, y_train, sample_weight=X_train['sex'].map(lambda x: 1 - eps if x == 'male' else eps))

normal_model = ExplainableBoostingClassifier(feature_names=X.columns.tolist())
normal_model.fit(X_train, y_train)

print("done")

done


In [4]:
foi = 'sex'
_x = X_train
_y = y_train

male_mask = _x[foi] == 'male'
female_mask = _x[foi] == 'female'

In [5]:
from interpretml_tools._old.interpretml_utils import GroupPerformanceAnalyzer
%matplotlib widget
plt.ioff()  # Avoids duplicate plots
analyzer = GroupPerformanceAnalyzer(
    male_model, female_model, normal_model,
    _x, _y,
    male_mask=male_mask, female_mask=female_mask,
    feature_of_interest='sex',
    combine_strategy='post',
    metric='log_likelihood',
)
analyzer.generate_plot(n_combinations=100)

Evaluating combinations:   0%|          | 0/100 [00:00<?, ?it/s]

Evaluating combinations: 100%|██████████| 100/100 [00:00<00:00, 104.23it/s]


AttributeError: 'GroupPerformanceAnalyzer' object has no attribute 'scatter_plots'

In [5]:
from interpretml_tools.interactive.new_gender_performance_curves import GenericGroupPerformanceAnalyzer

%matplotlib widget
plt.ioff()
analyzer = GenericGroupPerformanceAnalyzer(
    models_to_combine=[
        ("Male Model", male_model),
        ("Female Model", female_model),
        ("Normal Model", normal_model),
    ],
    baseline_models=[
    ],
    X_test=_x, y_test=_y,
    male_mask=male_mask, female_mask=female_mask,
    feature_of_interest='sex',
    metric='log_likelihood'
)
analyzer.generate_plot(n_combinations=100)

Processing Group 1/2:   0%|          | 0/100 [00:00<?, ?it/s]

Processing Group 2/2: 100%|██████████| 100/100 [00:00<00:00, 131.63it/s]

Plotting group: combination_group_0
x: [-0.3159692057125366, -0.3429185031011023, -0.4796549533122207, -0.33211842139655656, -0.45404926355885167, -0.31299106521965536, -0.3279597259501989, -0.31528568547131686, -0.35266747474588217, -0.3884908114546539, -0.39523065773654054, -0.3198316295134282, -0.43366252911419995, -0.36894646903724054, -0.3655894404546374, -0.39889753067030065, -0.36157400073329227, -0.36115486947551917, -0.3380279717177975, -0.42546390752451835, -0.3516400318584171, -0.33116575594985437, -0.4005295620551725, -0.32645368539129366, -0.31670191616782417, -0.38320640137895184, -0.3935337876068159, -0.3213688360548574, -0.3949284265081341, -0.45630187499111696, -0.4061757343535364, -0.3544711187584225, -0.3073753978594937, -0.40973945561469477, -0.3276679380080019, -0.47382080557992207, -0.3531866024244528, -0.46120365564551036, -0.28833870171078213, -0.48331139943568197, -0.4763667424637866, -0.41243545471752974, -0.38262922774137137, -0.46668984642793593, -0.33521455

# Adding more trained models

In [ ]:
import random

def generate_pairs(N, random_state=None):
    if random_state is not None:
        random.seed(random_state)
    pairs = [(random.uniform(0, 1), 0) for _ in range(N)]
    pairs = [(x, 1 - x) for x, _ in pairs]
    return pairs

In [ ]:
from tqdm.notebook import tqdm
import pickle

additional_models = []

for (mw, fw) in tqdm(generate_pairs(100, 42), desc="Training models"):
    new_model = ExplainableBoostingClassifier(feature_names=X.columns.tolist())
    # Create sample_weights based on sex
    sample_weights = X_train['sex'].map(lambda x: mw if x == 'male' else fw)

    # Fit the model with sample weights
    new_model.fit(X_train, y_train, sample_weight=sample_weights)


    # Add this model to our collection with the weights used
    additional_models.append((f"M: {mw:.2f}, F: {fw:.2f}", new_model))
    
    # Save the additional_models list to a pickle file
    with open("additional_models.pkl", "wb") as f:
        pickle.dump(additional_models, f)

Training models:   0%|          | 0/100 [00:00<?, ?it/s]

In [6]:
import pickle

with open("pickles/german/additional_models.pkl", "rb") as f:
    additional_models = pickle.load(f)

print(f"Loaded {len(additional_models)} models")

Loaded 50 models


In [ ]:
%matplotlib widget
plt.ioff()
analyzer = GenericGroupPerformanceAnalyzer(
    models_to_combine=[
        ("Male Model", male_model_eps),
        ("Normal Model", normal_model),
        ("Female Model", female_model_eps),
    ],
    baseline_models=additional_models,
    X_test=_x, y_test=_y,
    male_mask=male_mask, female_mask=female_mask,
    feature_of_interest='sex',
    metric='log_likelihood'
)
analyzer.generate_plot(n_combinations=100)

Processing Group 2/2: 100%|██████████| 100/100 [00:00<00:00, 131.45it/s]


Plotting group: combination_group_0
x: [-0.3765201959762147, -0.33153513765420006, -0.36537050995012565, -0.3908243950073008, -0.3942433373722212, -0.32986935747741974, -0.3662138734193226, -0.3050063478013948, -0.3581236863463744, -0.3636285079842418, -0.3535368893781719, -0.3288303092652437, -0.46470697161018915, -0.3837472402468054, -0.39029074184767854, -0.3836805691358039, -0.3530276799407324, -0.4072768795060514, -0.38237280260676587, -0.40808344022794274, -0.35750930214629345, -0.334209821588143, -0.43484747979318344, -0.38062489405408323, -0.4825451446079416, -0.32987078930339225, -0.4320016323769247, -0.32992893092753034, -0.34648849361161765, -0.33102462716368336, -0.4981519382291966, -0.34505179449952667, -0.4795489195174478, -0.35059026935682824, -0.34589002517202566, -0.3238409018485965, -0.3464360118012813, -0.31508171475121155, -0.3720601753484327, -0.4482900434186346, -0.3208583170426447, -0.3441264730356059, -0.3173278702828598, -0.4006922580211555, -0.3687927310131484

In [9]:
%matplotlib widget
plt.ioff()
analyzer = GenericGroupPerformanceAnalyzer(
    models_to_combine=[
        ("Male Model", male_model),
        ("Female Model", female_model),
        ("Normal Model", normal_model),
        *additional_models
    ],
    baseline_models=[],
    X_test=_x, y_test=_y,
    male_mask=male_mask, female_mask=female_mask,
    feature_of_interest='sex',
    metric='log_likelihood'
)
analyzer.generate_plot(n_combinations=100)

Evaluating All Models:   0%|          | 0/100 [00:00<?, ?it/s]

Evaluating All Models:   5%|▌         | 5/100 [00:05<01:38,  1.04s/it]


KeyboardInterrupt: 